In [1]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [11]:
!pip install transformers accelerate

In [2]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52 (from 1)
Receiving objects: 100% (52/52), 83.21 KiB | 655.00 KiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import random
random.seed(0)

In [6]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

**ベースラインモデルの評価**

In [22]:
import re

def run_chat_model(messages, max_new_tokens=256, do_sample=False):
    """共通のチャットモデル実行処理"""
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        eos_token_id=terminators,
        do_sample=do_sample,
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)


def generate_output(query):
    """質問に対する回答を生成"""
    return run_chat_model([{"role": "user", "content": query}])


def evaluate_yes_no(question, answer):
    """回答の質をYes/Noで評価"""
    prompt = f"質問: {question}\n回答: {answer}"
    evaluation = run_chat_model(
        [{"role": "user", "content": prompt}],
        max_new_tokens=50
    )
    return re.search(r'^yes', evaluation.strip().lower()) is not None


def generate_with_evaluation(question):
    """回答を生成し、その質を評価"""
    answer = generate_output(question)
    is_good = evaluate_yes_no(question, answer)
    return {
        "question": question,
        "answer": answer,
        "is_good": is_good,
        "evaluation": "Yes" if is_good else "No"
    }


In [23]:
question =  "モンテカルロ法がLLMの推論の性能を向上させることができるのか教えて"
response = generate_output(question)
print(response)

result = generate_with_evaluation(question)
print("\n評価結果:")
print(f"良い回答か: {result['evaluation']}")

 はい、モンテカルロ法はLLMの推論性能を向上させる可能性があります。 

**モンテカルロ法とは**

モンテカルロ法は、確率的探索法の一種で、未知の値を推定するために、ランダムな試行と評価の繰り返しを行う手法です。 

**LLMの推論におけるモンテカルロ法の活用**

1. **探索空間の効率的な探索:**  LLMは複雑なモデルであり、推論の過程で様々な可能性を考慮する必要があります。モンテカルロ法は、探索空間を効率的に探索し、より良い推論結果を得る可能性があります。
2. **データの解釈:**  モンテカルロ法は、データから学習したモデルの推論結果を解釈し、より正確な推論結果を得るのに役立ちます。
3. **複雑な問題の解決:**  複雑な問題を解決する際に、モンテカルロ法は、LLMの推論能力を向上させることで、より複雑な問題の解決に貢献します。

**具体的な例**

* **文章生成:**  モンテカルロ法を用いて、LLMがより自然

評価結果:
良い回答か: No


In [24]:
question =  "LangChainについて教えて"
response = generate_output(question)
print(response)

result = generate_with_evaluation(question)
print("\n評価結果:")
print(f"良い回答か: {result['evaluation']}")

## LangChain について

**LangChain** は、大規模言語モデル (LLM) を活用したアプリケーション開発を支援するオープンソースのフレームワークです。 

**主な特徴:**

* **LLM の連携:**  様々な LLM (例: ChatGPT, Bard, BLOOM) を統合し、複雑なタスクを実行できます。
* **データの管理:**  データのインデックス、検索、加工など、LLM の処理を効率的に行うためのツールを提供します。
* **アプリケーションの構築:**  アプリケーションの設計、開発、実行を支援します。
* **拡張性:**  独自の LLM を作成したり、既存の LLM を拡張したりできます。
* **オープンソース:**  コミュニティによる開発と拡張が活発です。

**LangChain を使うメリット:**

* **複雑なタスクの自動化:**  従来の LLM を使うと難しいタスクを、LangChain を使って簡単に実行できます。
* **効率的なデータ処理:**  データの管理と加工を自動化することで、開発時間を短縮できます。
* **柔軟なアプリケーション開発:**  様々な LLM と連携し、独自のアプリケーションを構築できます。
* **オープンソース

評価結果:
良い回答か: No


In [25]:
question =  "最近人気の生成AIツールについて３つ紹介をしてください。"
response = generate_output(question)
print(response)

result = generate_with_evaluation(question)
print("\n評価結果:")
print(f"良い回答か: {result['evaluation']}")

##  最近人気の生成AIツール３選

**1. ChatGPT (OpenAI)**
   - **特徴:**  自然な会話で文章を作成・編集できる、多様なタスクに対応できる強力なツール。
   - **用途:**  文章作成、要約、翻訳、質問応答、コード生成など、幅広い用途で活用されています。
   - **強み:**  人間に近い会話能力と、多様なタスクに対応できる柔軟性。

**2. DALL-E 2 (OpenAI)**
   - **特徴:**  画像を生成するAIツール。テキストを入力すると、そのテキストに基づいてオリジナルの画像を生成します。
   - **用途:**  イラスト、デザイン、写真、画像編集など、様々な場面で活用できます。
   - **強み:**  高画質で、複雑な画像生成が可能。

**3. Midjourney**
   - **特徴:**  ユーザーがテキストで画像を生成するAIツール。
   - **用途:**  イラスト、デザイン、写真、画像編集など、様々な場面で活用できます。
   - **強み:**  独特の雰囲気と、クリエイティブな表現力を特徴とする。


 




評価結果:
良い回答か: No


In [37]:
question =  "最近人気の生成AIツールについて３つ紹介をしてください。メリットに各ツールにつき１つずつ教えてください"
response = generate_output(question)
print(response)

result = generate_with_evaluation(question)
print("\n評価結果:")
print(f"良い回答か: {result['evaluation']}")

##  最近人気の生成AIツール３選

**1. ChatGPT (OpenAI)**
   * **メリット:**  自然な会話で質問や要約、文章作成など、様々なタスクに対応できる柔軟性が高い。

**2. DALL-E 2 (OpenAI)**
   * **メリット:**  画像生成能力が非常に高く、複雑なイメージやデザインを簡単に生成できる。

**3. Midjourney**
   * **メリット:**  アート作品のような高画質の画像を生成できる、クリエイティブな表現力を重視するユーザーに人気。



 




評価結果:
良い回答か: No


In [19]:
question =  "教育DX・GIGAスクール構想について、今後の方針について教えてください。あなたの考えも教えてください"
response = generate_output(question)
print(response)

result = generate_with_evaluation(question)
print("\n評価結果:")
print(f"良い回答か: {result['evaluation']}")

## 教育DX・GIGAスクール構想の今後の方針

**教育DX・GIGAスクール構想**は、教育の質・量を向上させるために、デジタル技術を活用した教育システムの構築を目指しています。 

**今後の方針としては、以下の点が挙げられます。**

1. **デジタル技術の普及と活用:**
    * **教育アプリ・ツールへの積極的な導入:**  学習内容の多様化、学習環境の自由化、学習者の個別指導の促進を図る。
    * **教育現場におけるデジタルスキル向上:**  教師・生徒のデジタルスキル向上を促進し、デジタル技術を活用した教育活動の効率化と質の向上を図る。
    * **教育データの活用:**  学習状況や生徒の個性に応じた個別指導や学習内容の最適化を可能にする。
    * **教育プラットフォームの構築:**  教育機関間や地域間での連携を促進し、教育の質向上と地域社会への貢献を図る。

2. **教育の質・量向上:**
    * **学習内容の多様化:**  デジタル技術を活用した学習コンテンツの提供により、多様な学習スタイルに対応する。

評価結果:
良い回答か: No


RAG

In [38]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [39]:
with open("/content/lecture-ai-engineering/day3/data/question1.txt", "r") as f:
  raw_writedown = f.read()

In [41]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))

ドキュメントサイズ:  71


In [44]:
# Retrievalの実行
question = "モンテカルロ法がLLMの推論の性能を向上させることができるのか教えて"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[74.99674987792969, 75.32292938232422, 74.23294830322266, 75.86393737792969, 74.68770599365234, 70.9331283569336, 74.18632507324219, 78.26974487304688, 75.48431396484375, 72.72064971923828, 74.24675750732422, 65.84030151367188, 64.95155334472656, 61.84282302856445, 70.5948715209961, 67.68824005126953, 74.66991424560547, 66.9796371459961, 71.58440399169922, 69.94791412353516, 66.09217834472656, 70.09321594238281, 63.795650482177734, 69.16935729980469, 72.66394805908203, 69.48240661621094, 73.3190689086914, 71.82195281982422, 69.59042358398438, 72.53857421875, 67.9404525756836, 71.13493347167969, 77.7258071899414, 75.6706314086914, 76.69139862060547, 74.99674987792969, 75.32292938232422, 74.23294830322266, 75.86393737792969, 74.68770599365234, 70.9331283569336, 74.18632507324219, 78.26974487304688, 75.48431396484375, 72.72064971923828, 74.24675750732422, 65.84030151367188, 64.95155334472656, 61.84282302856445, 70.5948715209961, 67.68824005126953, 74.66991424560547, 66.9796371459961, 71.

In [45]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 78.26974487304688)
A*探索やモンテカルロ木探索 (MCTS) も、複雑な推論問題を解決する際のLLMの性能を向上させる計画技術として応用されている 


取得したドキュメント2: (Score: 78.26974487304688)
A*探索やモンテカルロ木探索 (MCTS) も、複雑な推論問題を解決する際のLLMの性能を向上させる計画技術として応用されている 


取得したドキュメント3: (Score: 77.7258071899414)
まとめ
本論文では、LLMの多段階推論能力を向上させるための新しいフレームワークQが提案されている 


取得したドキュメント4: (Score: 77.7258071899414)
まとめ
本論文では、LLMの多段階推論能力を向上させるための新しいフレームワークQが提案されている 


取得したドキュメント5: (Score: 76.69139862060547)
実験結果から、Q*は既存の手法と比較して優れた性能を示しており、LLMの多段階推論における重要な進歩と言える 




In [48]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] モンテカルロ法がLLMの推論の性能を向上させることができるのか教えて"
response = generate_output(query)
print(response)

result = generate_with_evaluation(question)
print("\n評価結果:")
print(f"良い回答か: {result['evaluation']}")

はい、**モンテカルロ木探索 (MCTS)** は LLM の推論の性能を向上させる可能性があります。

**モンテカルロ木探索** は、複雑な推論問題を解決する際に、**探索と学習** を組み合わせた計画技術です。LLM の多段階推論能力を向上させるには、**探索** を用いて、**学習** を用いて最適な行動を選択するようなアプローチが考えられます。

**モンテカルロ木探索のメリット**

* **複雑な状況に対応:**  複雑な推論問題を解決する際に、従来の探索方法では困難な状況に対応できます。
* **学習の効率性:**  MCTS は、過去の経験から学習し、より良い行動を選択する能力を持っています。
* **柔軟性:**  MCTS は、様々な種類の推論問題に対応できます。


**ただし、MCTS の導入にはいくつかの課題も存在します。**

* **計算コスト:**  MCTS は、複雑な推論問題を解決する際に、計算コストがかかります。
* **初期化の難しさ:**  MCTS の初期化は、複雑な推論問題を解決する際に、難しい場合があります

評価結果:
良い回答か: No


In [49]:
with open("/content/lecture-ai-engineering/day3/data/question2.txt", "r") as f:
  raw_writedown = f.read()

In [50]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))

ドキュメントサイズ:  50


In [52]:
# Retrievalの実行
question = "LangChainについて教えて"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[86.68946838378906, 73.36783599853516, 69.24925994873047, 84.7535400390625, 82.85494232177734, 82.86796569824219, 84.38323974609375, 84.9334716796875, 76.15711975097656, 74.01545715332031, 69.59127044677734, 68.39466094970703, 69.16783142089844, 69.48277282714844, 73.82241821289062, 71.62269592285156, 71.97805786132812, 69.63578796386719, 66.49281311035156, 70.97042846679688, 67.63601684570312, 67.79962158203125, 85.20816040039062, 83.83651733398438, 84.91825866699219, 85.64728546142578, 76.40245056152344, 66.37544250488281, 69.38652038574219, 72.92601776123047, 83.9635238647461, 85.4390640258789, 81.30380249023438, 68.27350616455078, 71.5831069946289, 69.50020599365234, 85.67391967773438, 73.20600128173828, 66.80066680908203, 70.90939331054688, 83.58757781982422, 69.33911895751953, 70.27019500732422, 83.2523193359375, 69.42546081542969, 68.83455657958984, 69.37289428710938, 80.885986328125, 69.78370666503906, 61.96743392944336]]


In [53]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 86.68946838378906)
LangChain は、大規模言語モデル (LLM) に基づいてアプリケーションを構築するためのオープンソースフレームワークです 


取得したドキュメント2: (Score: 85.67391967773438)
エージェント
開発者は、LangChain が提供するツールとライブラリを使用して、複雑なアプリケーション向けに既存のチェーンを構成およびカスタマイズします 


取得したドキュメント3: (Score: 85.64728546142578)
pip install langchain

次に、開発者はチェーンビルディングブロックまたは LangChain 表現言語 (LCEL) を使用して、簡単なプログラミングコマンドでチェーンを構成します 


取得したドキュメント4: (Score: 85.4390640258789)
LLM インターフェイス
LangChain は、開発者がコードから LLM に接続してクエリを実行できる API を提供します 


取得したドキュメント5: (Score: 85.20816040039062)
LangChain フレームワークでは、リンクはユーザーからの入力を受け取り、それを LangChain ライブラリに渡して処理します 




In [54]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] LangChainについて教えて"
response = generate_output(query)
print(response)

result = generate_with_evaluation(question)
print("\n評価結果:")
print(f"良い回答か: {result['evaluation']}")

## LangChain について

LangChain は、大規模言語モデル (LLM) を活用したアプリケーション開発のためのオープンソースフレームワークです。 

**簡単に言うと、LangChain を使うと、複雑なアプリケーションを、LLM を中心としたシンプルな構成で構築できます。**

**主な特徴は以下の通りです:**

* **チェーンビルディング:**  LangChain は、複雑なアプリケーションを、複数の LLM への接続や処理を組み合わせた「チェーン」で構築できます。
* **柔軟性:**  開発者は、独自の LLM 接続や処理方法を自由にカスタマイズできます。
* **使いやすいインターフェース:**  LangChain は、LLM に接続してクエリを実行するための API を提供し、開発者がコードから LLM に接続する手間を省きます。
* **使いやすいツール:**  LangChain は、チェーンビルディングブロックや LangChain 表現言語 (LCEL) を提供し、プログラミングコマンドで複雑なアプリケーションを構築できます。

**具体的な機能:**

* **LLM インターフェース:**  LLM に直接接続し、クエリを実行できます。
* **リンク:**  ユーザーからの入力を受け取り、処理します。
* **チェーンビルディング

評価結果:
良い回答か: No


In [55]:
with open("/content/lecture-ai-engineering/day3/data/question3.txt", "r") as f:
  raw_writedown = f.read()

# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))

# Retrievalの実行
question = "最近人気の生成AIツールについて３つ紹介をしてください。"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 最近人気の生成AIツールについて３つ紹介をしてください。"
response = generate_output(query)
print(response)

result = generate_with_evaluation(question)
print("\n評価結果:")
print(f"良い回答か: {result['evaluation']}")

ドキュメントサイズ:  99
[[87.02513122558594, 84.64681243896484, 83.41026306152344, 81.27604675292969, 85.93749237060547, 80.73993682861328, 80.43097686767578, 71.6632080078125, 71.29732513427734, 80.75189971923828, 78.9420166015625, 73.88890838623047, 79.57788848876953, 77.73470306396484, 79.4071044921875, 75.13944244384766, 80.19373321533203, 86.66348266601562, 84.93363952636719, 85.41825866699219, 79.90957641601562, 76.15729522705078, 77.07122802734375, 75.59828186035156, 80.9507827758789, 82.83905029296875, 75.43357849121094, 74.29380798339844, 71.4930191040039, 75.550537109375, 73.00044250488281, 71.52609252929688, 70.32275390625, 80.18270874023438, 76.67211151123047, 70.61693572998047, 67.70201110839844, 77.67448425292969, 82.01556396484375, 75.00922393798828, 80.96149444580078, 73.89434814453125, 81.03865051269531, 79.56735229492188, 73.06344604492188, 85.40956115722656, 85.35514831542969, 79.79457092285156, 72.77369689941406, 79.38240051269531, 76.96734619140625, 81.32330322265625, 78.55

In [57]:
with open("/content/lecture-ai-engineering/day3/data/question3.txt", "r") as f:
  raw_writedown = f.read()

# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))

# Retrievalの実行
question = "最近人気の生成AIツールについて３つ紹介をしてください。その際に活用方法について１つ教えてください"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 最近人気の生成AIツールについて３つ紹介をしてください。その際に活用方法について１つ教えてください"
response = generate_output(query)
print(response)

result = generate_with_evaluation(question)
print("\n評価結果:")
print(f"良い回答か: {result['evaluation']}")

ドキュメントサイズ:  99
[[86.11143493652344, 85.08016204833984, 83.29364776611328, 79.55691528320312, 84.51290130615234, 79.60855102539062, 79.89936065673828, 70.44355010986328, 70.30972290039062, 79.31903839111328, 77.24435424804688, 72.37356567382812, 77.96902465820312, 76.87106323242188, 77.97171783447266, 74.46739196777344, 79.25225067138672, 85.53952026367188, 83.30905151367188, 83.48568725585938, 78.5782241821289, 75.5633773803711, 76.13182067871094, 75.4296646118164, 80.0523910522461, 81.31573486328125, 73.69556427001953, 73.00751495361328, 70.0781021118164, 73.97930908203125, 71.77291107177734, 70.16275787353516, 69.94196319580078, 79.01261901855469, 75.1622543334961, 70.02359008789062, 67.38562774658203, 76.74555969238281, 80.609375, 73.5987548828125, 80.04769897460938, 72.66053009033203, 79.33956909179688, 77.99099731445312, 71.6505126953125, 83.68763732910156, 83.68492126464844, 78.46464538574219, 71.43861389160156, 78.23793029785156, 76.00184631347656, 79.94803619384766, 77.49667358

In [62]:
with open("/content/lecture-ai-engineering/day3/data/question5.txt", "r") as f:
  raw_writedown = f.read()

# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))

# Retrievalの実行
question = "教育DX・GIGAスクール構想について、今後の方針について教えてください。"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 教育DX・GIGAスクール構想について、今後の方針について教えてください。"
response = generate_output(query)
print(response)

result = generate_with_evaluation(question)
print("\n評価結果:")
print(f"良い回答か: {result['evaluation']}")

ドキュメントサイズ:  9
[[82.23751068115234, 65.50576782226562, 75.10757446289062, 76.9798583984375, 74.65925598144531, 76.93286895751953, 66.4249496459961, 76.6470947265625, 52.57961654663086]]
取得したドキュメント1: (Score: 82.23751068115234)
令和6年度教育DX・GIGAスクール構想

GIGAスクールにおける学びの充実
個別最適な学びと協働的な学びの一体的な充実など、教育の質を向上させるために、「GIGAスクール構想」の下で児童生徒の１人１台端末及び通信ネットワー
ク等の学校ICT環境での新しい学びが本格的に開始されている 


取得したドキュメント2: (Score: 76.9798583984375)
また、GIGAスクール構想第２ステージに向けては、準備が整った自治体・学校において生成AIの適
切な活用や高度なプログラミング教育、デジタルものづくりなどの先進事例を創出する必要がある 


取得したドキュメント3: (Score: 76.93286895751953)
〇児童生徒の情報活用能力の把握に関する調査研究
●調査問題の妥当性等を検証するための予備調査の検証など
●次回本調査に向けた準備・実施
学校DX戦略アドバイザー リーディングDXスクール
令和6年度予算額（案） 3億円
（前年度予算額 3億円）
（担当：初等中等教育局修学支援・教材課）
実施内容②
・生成AIを活用した授業実践研究 ※ガイドラインを遵守 ※効果的な
取組実践を創出する観点から、学術的知見を有する研究者や優れた
実践家等から伴走支援を受けること（学校DX戦略アドバイザーの支
援含む）
・生成AI を用いた取組の成果に関し、年度末に実施する成果報告会
で発表（予定）

9
GIGAスクール構想の推進
～1人1台端末の着実な更新～ 令和5年度補正予算額 2,661億円
現状・課題
事業内容・スキーム
 全ての子供たちの可能性を引き出す個別最適な学びと協働的な学びを実現するため、令和２～３年度に「1人1台端末」と高速通信
ネットワークを集中的に整備し、ＧＩＧＡスクール構想を推進 

